In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Exercise 8e.

In [2]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

def split_train_valid(data, split_number=5000):
    return data[split_number:], data[:split_number]

x_train, x_valid = split_train_valid(x_train_full)
y_train, y_valid = split_train_valid(y_train_full)

x_means = x_train.mean(axis=0)
x_stds = x_train.std(axis=0)
x_train_scaled = (x_train - x_means) / x_stds
x_valid_scaled = (x_valid - x_means) / x_stds
x_test_scaled = (x_test - x_means) / x_stds

In [3]:
def make_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
    for i in range(20):
        model.add(keras.layers.Dense(100, 
                                     activation='selu', 
                                     kernel_initializer='lecun_normal'))
    model.add(keras.layers.AlphaDropout(rate=0.1))
    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

In [4]:
loss = 'sparse_categorical_crossentropy'
optimizer = keras.optimizers.Nadam(learning_rate=1e-4)
n_epochs = 100
early_stopping_cb = keras.callbacks.EarlyStopping(patience=25)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('alpha_dropout_cifar10_model.h5', save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]

In [5]:
model = make_model()
model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

history = model.fit(x_train_scaled, y_train, 
                    epochs=n_epochs, 
                    validation_data=(x_valid_scaled, y_valid), 
                    callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 11s 7ms/step - loss: 1.9330 - acc: 0.3157 - val_loss: 1.6867 - val_acc: 0.4024
Epoch 2/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.6391 - acc: 0.4175 - val_loss: 1.6027 - val_acc: 0.4320
Epoch 3/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.5274 - acc: 0.4576 - val_loss: 1.5454 - val_acc: 0.4550
Epoch 4/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4474 - acc: 0.4865 - val_loss: 1.5052 - val_acc: 0.4630
Epoch 5/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.3830 - acc: 0.5080 - val_loss: 1.5044 - val_acc: 0.4836
Epoch 6/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.3294 - acc: 0.5268 - val_loss: 1.4900 - val_acc: 0.4902
Epoch 7/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.2780 - acc: 0.5451 - val_loss: 1.5006 - val_acc: 0.4850
Epoch 8/100
1407/1407 [==============================]

In [6]:
model = keras.models.load_model('alpha_dropout_cifar10_model.h5')
model.evaluate(x_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.4858 - acc: 0.5078


[1.4858381748199463, 0.5077999830245972]

In [7]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [8]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [9]:
def mc_dropout_predict_probas(mc_model, x, n_samples=10):
    y_probas = [mc_model.predict(x) for sample in range(n_samples)]
    return np.mean(y_probas, axis=0)


def mc_dropout_predict_classes(mc_model, x, n_samples=10):
    y_probas = mc_dropout_predict_probas(mc_model, x, n_samples)
    return np.argmax(y_probas, axis=1)

In [10]:
y_pred = mc_dropout_predict_classes(mc_model, x_valid_scaled)
acc = np.mean(y_pred == y_valid[:, 0])
acc

0.5084